In [13]:
import numpy as np
import re
import pandas as pd

In [14]:
df = pd.read_csv('2018-04.csv', header=None, names=['data'])
df['datetime'] = df['data'].str.extract(r'(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})')
df['datetime'] = df['datetime'].str.replace('1518','2000')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M')
df.sort_values(by=['datetime'], inplace=True)

df['date'] = df['data'].str.extract(r'(\d{4}-\d{2}-\d{2})')
df['date'] = df['date'].str.replace('1518','2000')
df['time'] = df['data'].str.extract(r'(\d{2}:\d{2})')
df['min'] = df['data'].str.extract(r':(\d{2})]')
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df['guard_id'] = df['data'].str.extract(r'#(\d+)')
df['guard_id'] = df['guard_id'].fillna(method='ffill')
df['action'] = df['data'].str.extract(r'(begins shift|falls asleep|wakes up)')

df['duration'] = np.where((df.guard_id == df.guard_id.shift())&(df.action != 'begins shift'), df.datetime - df.datetime.shift(), np.nan)
df['duration_min'] = df['duration'] / np.timedelta64(1, 'm')

df['min'] = df['min'].astype(np.int64)
df['duration_min'] = df['duration_min'].fillna(0)
df['duration_min'] = df['duration_min'].astype(np.int64)

df.drop(columns=['data'], inplace=True)

df.head()

,datetime,date,time,min,guard_id,action,duration,duration_min
479,2000-02-15 23:58:00,2000-02-15,23:58,58,3557,begins shift,NaT,0
958,2000-02-16 00:35:00,2000-02-16,00:35,35,3557,falls asleep,00:37:00,37
124,2000-02-16 00:38:00,2000-02-16,00:38,38,3557,wakes up,00:03:00,3
375,2000-02-16 00:43:00,2000-02-16,00:43,43,3557,falls asleep,00:05:00,5
467,2000-02-16 00:45:00,2000-02-16,00:45,45,3557,wakes up,00:02:00,2


In [15]:
df_pivot = df[df['action']=='wakes up'].pivot_table(values='duration_min', index='guard_id', columns=None, aggfunc='sum')
df_pivot = df_pivot.reindex(df_pivot.sort_values(by='duration_min', ascending=False).index)
most_asleep_guard_id = list(df_pivot.index)[0]
df_pivot.head(1)

,duration_min
guard_id,
3457,504


In [16]:
df_sleep_mins = pd.DataFrame({'sleep_time':np.zeros((60))})
df_sleep_mins.index.name = 'minute'

In [17]:
for index, row in df[df['guard_id'] == most_asleep_guard_id].iterrows():
    if row['action'] == 'wakes up':
        df_sleep_mins.iloc[row['min']-row['duration_min']:row['min'],0] += 1

In [18]:
df_sleep_mins.iloc[df_sleep_mins[['sleep_time']].idxmax()]

,sleep_time
minute,
40,14.0


In [19]:
most_slept_min = df_sleep_mins.iloc[df_sleep_mins[['sleep_time']].idxmax()].index.values

In [20]:
print('Part 1 answer:', int(most_asleep_guard_id)*int(most_slept_min))

Part 1 answer: 138280


# PART TWO

In [21]:
df_by_guards = pd.DataFrame(np.zeros((60,len(df['guard_id'].unique()))), columns = df['guard_id'].unique())
df_by_guards.head()

,3557,3457,2819,1697,1901,2459,743,443,2593,1693,...,1733,3559,571,1627,1151,101,3301,2657,673,827
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
for index, row in df.iterrows():
    if row['action'] == 'wakes up':
        df_by_guards[row['guard_id']].loc[row['min']-row['duration_min']:row['min']-1] += 1

In [23]:
guard_id = list(df_by_guards.max().sort_values(ascending=False).head(1).index)[0]
sleep_count = df_by_guards.max().sort_values(ascending=False).head(1)[0]
top_minute = df_by_guards.index[df_by_guards[guard_id] == sleep_count][0]

In [24]:
print('Part 2 answer:', int(guard_id)*top_minute)

Part 2 answer: 89347
